In [1]:
import cx_Oracle
from IPython.display import IFrame    #to display pdf file
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pickle
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
# Load the data
data_dir = 'Data/'
Stores = pd.read_csv(data_dir + 'AllStoresKansas.csv', dtype = str)
StoreType = pd.read_csv(data_dir + 'NonLowPointStoresKansas.csv', dtype = str)
KansasCounties = pd.read_csv(data_dir + 'KansasCounties.csv', dtype = str)
Stores.head()

,RTL_STORE_CD,RTL_FIPS_COUNTY_DSC,RTL_PREMISE_TYPE_CD,RTL_CHANNEL_DSC,RTL_SUBCHANNEL_DSC,RTL_BEER_FLAG,RTL_LIQUOR_FLG
0,101403888,SEDGWICK,OFF,LIQUOR,CONVENTIONAL LIQUOR,Y,Y
1,100091604,SALINE,OFF,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,Y,N
2,200697901,SEDGWICK,OFF,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,Y,N
3,100091143,HARPER,OFF,GROCERY,SUPERMARKET-CONVENTIONAL,Y,N
4,107381463,DOUGLAS,OFF,GROCERY,SUPERMARKET-NATURAL/GOURMET FOODS,Y,N


In [3]:
Stores['RTL_STATE_DSC'] = 'KANSAS'
Stores['LowPoint_Y'] = ''
Stores.columns

Index([u'RTL_STORE_CD', u'RTL_FIPS_COUNTY_DSC', u'RTL_PREMISE_TYPE_CD',
       u'RTL_CHANNEL_DSC', u'RTL_SUBCHANNEL_DSC', u'RTL_BEER_FLAG',
       u'RTL_LIQUOR_FLG', u'RTL_STATE_DSC', u'LowPoint_Y'],
      dtype='object')

In [4]:
KansasCounties.head()

,COUNTY,STATE,STATUS
0,ALLEN,KANSAS,MIXED
1,ANDERSON,KANSAS,MIXED
2,ATCHISON,KANSAS,MIXED
3,BARBER,KANSAS,WET
4,BARTON,KANSAS,WET


#convert the column name into capitalized 
StoreType.columns = [x.upper() for x in StoreType.columns]
Stores.columns = [x.upper() for x in Stores.columns]
KansasCounties.columns = [x.upper() for x in KansasCounties.columns]

In [5]:
FullData = pd.merge(Stores, StoreType, on="RTL_STORE_CD", how="left")
FullData.loc[FullData['BEERTYPE'].isnull(),"BEERTYPE"] = "LowPoint"
FullData = pd.concat([FullData, pd.get_dummies(FullData['RTL_FIPS_COUNTY_DSC'],prefix='COUNTY')], axis=1)
FullData = pd.concat([FullData, pd.get_dummies(FullData['RTL_PREMISE_TYPE_CD'],prefix='PREMISE')], axis=1)
FullData = pd.concat([FullData, pd.get_dummies(FullData['RTL_CHANNEL_DSC'],prefix='CHANNEL')], axis=1)
FullData = pd.concat([FullData, pd.get_dummies(FullData['RTL_SUBCHANNEL_DSC'],prefix='SUBCHANNEL')], axis=1)
FullData = pd.concat([FullData, pd.get_dummies(FullData['RTL_BEER_FLAG'],prefix='BEER_LICENSE')], axis=1)
FullData = pd.concat([FullData, pd.get_dummies(FullData['RTL_LIQUOR_FLG'],prefix='LIQUOR_LICENSE')], axis=1)
FullData.head()

,RTL_STORE_CD,RTL_FIPS_COUNTY_DSC,RTL_PREMISE_TYPE_CD,RTL_CHANNEL_DSC,RTL_SUBCHANNEL_DSC,RTL_BEER_FLAG,RTL_LIQUOR_FLG,RTL_STATE_DSC,LowPoint_Y,BEERTYPE,...,SUBCHANNEL_SUPERMARKET-NATURAL/GOURMET FOODS,SUBCHANNEL_THEATER,SUBCHANNEL_UNKNOWN,SUBCHANNEL_WINE SPECIALTY STORE,BEER_LICENSE_N,BEER_LICENSE_U,BEER_LICENSE_Y,LIQUOR_LICENSE_N,LIQUOR_LICENSE_U,LIQUOR_LICENSE_Y
0,101403888,SEDGWICK,OFF,LIQUOR,CONVENTIONAL LIQUOR,Y,Y,KANSAS,,NonLowPoint,...,0,0,0,0,0,0,1,0,0,1
1,100091604,SALINE,OFF,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,Y,N,KANSAS,,LowPoint,...,0,0,0,0,0,0,1,1,0,0
2,200697901,SEDGWICK,OFF,CONVENIENCE STORE,CONVENTIONAL CONVENIENCE,Y,N,KANSAS,,LowPoint,...,0,0,0,0,0,0,1,1,0,0
3,100091143,HARPER,OFF,GROCERY,SUPERMARKET-CONVENTIONAL,Y,N,KANSAS,,LowPoint,...,0,0,0,0,0,0,1,1,0,0
4,107381463,DOUGLAS,OFF,GROCERY,SUPERMARKET-NATURAL/GOURMET FOODS,Y,N,KANSAS,,LowPoint,...,1,0,0,0,0,0,1,1,0,0


In [7]:
#save the columns in pickle
columns = list(Stores.columns)

raw_columns = 'classification_columns.p'
pickle.dump(columns, open('classification_columns.p', 'wb'))

# Function for production whole

In [ ]:
#save this one 
def classify_32(self, state, isPickle):
    
    '''
    This function takes in dataframe, and classify the store type to either it is a LowPoint store or not.
        LowPoint_Y = True: stores that can only sell 3.2% beer.
        LowPoint_Y = False: stores that can sell any beer.
    
    Parameter:
    ---------
        state: takes a string. Indicates the state.
        isPickle: takes a boolean. Indicates classification method. 'True' for pickle, 'False' for grid search
        
    Return:
    -------
        the dataframe with prediction result under 'LowPoint_Y' column.
        
    '''
    
    #define a function to get the subset of dataframe
    def classification_subset(self, state):

        '''
        This function subsets the dataframe to prepare classification.

        parameter:
        ---------
            state: Takes a string. The name of state to subset dataframe. 
            
        return:
        -------
            The subsetted dataframe.        
        '''

        #load the columns for subset from pickle:
        subset_columns = pickle.load(open('classification_columns.p', 'rb'))

        #subset the dataframe:
        classification_df = pd.DataFrame(self.data[subset_columns])

        classification_df = classification_df[classification_df['RTL_STATE_DSC'] == str(state).upper()]

        
    
    #define a function to run pickle for decision tree model. 
    def decision_tree_pickle(self, classification_df, state):

        '''
        This function takes the name of the state, and using the subsetted dataframe from function 'classification_subset' 
            to classify the stores using pickled decision tree model. 

        parameter:
        ----------
            classification_df: Takes a dataframe. The subsetted dataframe after excuting the function "classification_subset"
            state: Takes a string. The name of the State

        return:
        -------
            The dataframe with prediction result.        
        '''
        #call the binarize function to transform the columns into binary
        result_df = self.binarize([c for c in classification_df.columns if c not in ['RTL_STORE_CD',"LowPoint_Y"]])

        #load the model from pickle for that state
        filename = str(state).title()+'_decision_tree_model.p'
        decision_tree_model = pickle.load(open(filename, 'rb'))

        #prepare independent variables:
        X_labels = [c for c in result_df.columns if c not in [classification_df.columns]]
        X = result_df.loc[:,X_labels]

        #dependent variable:
        result_df['LowPoint_Y'] = decision_tree_model.predict(X) == 'LowPoint'
        
    #define a function to run pickle for business rule. 
    def business_rule_pickle(self, classification_df, state):
        '''
        This function takes the name of the state, and using the subsetted dataframe from function 'classification_subset' 
            to classify the stores using pickled business rule.

        parameter:
        ----------
            state: Takes a string. The name of the State
            classification_df: Takes a dataframe. The subsetted dataframe after excuting the function "classification_subset"

        return:
        -------
            The dataframe with prediction result.        
        '''    

        #load the business rule from pickle for that State
        filename = str(state).title()+'_business_rule.p'
        rule_func = pickle.load(open('filename.p', 'rb'))

        #make the prediction
        rule_func(classification_df)

    def read_query_and_predict(self, classification_df, state):
    
        '''
        This function queries from Oracle SQL for specific state and store type and save the data in a dataframe. Then, use decision
            tree classifier to train the labeled data, and make prediction on whole dataset.

        parameter:
        ---------
            classification_df: Takes a dataframe. The subsetted dataframe after excuting the function "classification_subset"
            state: Takes a string. The state name

        return:
        -------
            a dataframe with prediction result. 
        '''


        #Access to Oracle DB
        host = 'tncluster6.cbi.net'
        port = 1521
        s_name = 'EDW1TS_EX.cbi.net'
        dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
        db_ts = cx_Oracle.connect('USERNAME', 'PASSWORD', dsn_tns)
        cursor = db_ts.cursor()

        if str(state) == 'Kansas'.casefold():
            state_code = str(162)
        elif str(state) == 'Utah'.casefold():
            state_code = str(239)
        elif str(state) == 'Minnesota'.casefold():
            state_code = str(177)
        elif str(state) == 'Corolado'.casefold():
            state_code = str(125)
        elif str(state) == 'Oklahoma'.casefold():
            state_code = str(201)

        query = '''select /*+ parallel(6) */ a15.STORE_CD  RTL_STORE_CD, 'NonLowPoint' as "BEERTYPE"
                    from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
                    join EDW.BI_CRN_D_ITEM_VW a12
                       on  (a11.ITEM_ID = a12.ITEM_ID)
                     join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a13
                       on  (a11.DATE_ID = a13.REL_DATE_ID)
                     join EDW.BI_CRN_D_DISTRIBUTOR_VW a14
                       on  (a11.DIST_ID = a14.DIST_ID)
                     join EDW.BI_CRN_D_RETAILER_VW a15
                       on  (a11.RETAILER_ID = a15.RETAILER_ID)
                    where (a13.REL_TIME_CD in ('L3_TY')
                     and a14.STATE_PROVINCE_COUNTRY_CD in (''' + "'" + state_code + "')" + ''' and a12.MASTER_SKU_CD not in ('80013450', '80013452', '80013455', '80013979', '80013456', '80013475', '80061325', '80013477', '80013982', '80013980', '80059842', '80058839', '80014020', '80014022', '80014023', '80014024', '80014025', '80013992', '80013991', '80013464', '80013466', '80013469', '80013471', '80013472', '80013473', '80013478', '80031998', '80013485', '80056926', '80058838', '80019746', '80031994', '80015966', '80014017', '80012704', '80014015', '80014009', '80014011', '80024732', '80013994', '80056172', '80014010'))
                    group by a15.STORE_CD
                    UNION 
                    select /*+ parallel(6) */ a15.STORE_CD  RTL_STORE_CD,
                     'LowPoint' as "BEERTYPE"
                    from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
                     join EDW.BI_CRN_D_ITEM_VW a12
                       on  (a11.ITEM_ID = a12.ITEM_ID)
                     join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a13
                       on  (a11.DATE_ID = a13.REL_DATE_ID)
                     join EDW.BI_CRN_D_DISTRIBUTOR_VW a14
                       on  (a11.DIST_ID = a14.DIST_ID)
                     join EDW.BI_CRN_D_RETAILER_VW a15
                       on  (a11.RETAILER_ID = a15.RETAILER_ID)
                    where (a13.REL_TIME_CD in ('L3_TY')
                     and a14.STATE_PROVINCE_COUNTRY_CD in (''' +"'" + state_code + "')" + ''' and a12.MASTER_SKU_CD not in ('80029020', '80060325', '80032234', '80032233', '80023100', '80031072', '80013435', '80013437', '80013439', '80057078', '80014016', '80013442', '80026873', '80056799', '80013444', '80014014', '80013447', '80013460', '80013457', '80013458', '80059543', '80013461', '80059542', '80059571', '80059572', '80059573', '80013982', '80013980', '80056922', '80014002', '80013968', '80029050', '80013515', '80013516', '80013517', '80013518', '80018933', '80056908', '80013520', '80014006', '80013993', '80013522', '80014026', '80013984', '80013978', '80013970', '80014012', '80014060', '80013986', '80013977', '80060330', '80060331', '80060328', '999', '80014008', '80013995', '80013972', '80056887', '80014001', '80031989', '80013985', '80013971', '80013983', '80014007', '80029021', '80015214', '80015215', '80014005', '80014004', '80013976', '80013990', '80013998', '80013989', '80013975', '80013997', '80027630', '80027631', '80027632', '80062390', '80062550', '80013996', '80014013', '80013988', '80013987', '80013974', '80013973', '80013999', '80014000', 'UNK', '80013981'))
                    group by a15.STORE_CD '''

        try:
            cursor.execute(query)
            names = [ x[0] for x in cursor.description]
            rows = cursor.fetchall()
            StoreType = pd.DataFrame( rows, columns=names)

        finally:
            if cursor is not None:
                cursor.close()

        #If the above code runs sucessfully, it will return a dataframe which columns: store number and store type(LowPoint or NonLowPoint)



        #left join the store type data with subsetted dataframe
        result_df = pd.merge(classification_df, StoreType, on="RTL_STORE_CD", how="left")  

        #call the binarize function to transform the columns into binary
        result_df = self.binarize([c for c in classification_df.columns if c not in ['RTL_STORE_CD',"LowPoint_Y"]])

        #subset the training dataset
        train_df = result_df.loc[result_df['BEERTYPE'] != '']  

        #fit the model using traning data
        #independent variables:
        X_labels = [c for c in train_df.columns if c not in [classification_df.columns]]
        X_train = train_df.loc[:,X_labels]
        #dependent variable:
        Y_train = train_df.loc['BEERTYPE'] 

        #train the model, and find the best parameter
        parameters = {'max_depth':range(1,21), 'min_samples_leaf':range(3,21,3), 'min_samples_split':range(3,21,3), 'random_state': [0]}
        clf = tree.DecisionTreeClassifier()
        clf = GridSearchCV(clf, parameters, n_jobs = -2)
        clf.fit(X_train, Y_train)
        accuracy = clf.best_score_ 
        best_params = clf.best_params_
        best_depth = best_params['max_depth']
        best_leaf = best_params['min_samples_leaf']
        best_split = best_params['min_samples_split']

        #using the best parameter to train and fit the model
        clf = tree.DecisionTreeClassifier(max_depth=best_depth, min_samples_leaf= best_leaf, min_samples_split = best_split, random_state=0)
        clf = clf.fit(X_train,Y_train)

        #Prepare for prediction dataset
        X_pred = result_df.loc[:,X_labels]

        #make prediction for whole dataset.
        result_df['LowPoint_Y'] = clf.predict(X_pred) == 'LowPoint_Y'        

        #delete the column 'BEERTYPE'
        del result_df['BEERTYPE']    
     
    
    
    ###############################excute code########################
    
    #create a LowPoint_Y column for entire dataset. 
    self.data['LowPoint_Y'] = ''
    
    #take a subset of the data classification_subset 
    classification_df = self.classification_subset(self.data)
    
    #if using Pickle
    if isPickle:
        
        #Decision tree model for Kansas and Minnesota
        #If the state is Kansas:
        if state == 'Kansas'.casefold():
            self.decision_tree_pickle(classification_df, 'Kansas')

        #If the state is Minnesota:   
        elif state == 'Minnesota'.casefold():
            self.decision_tree_pickle(classification_df, 'Minnesota')
      
    
        #Business rule for Utah, Oklahoma, and Colorado
        #If the state is Utah:
        elif state == 'Utah'.casefold():
            self.business_rule_pickle(classification_df, 'Utah')
            
        #If the state is Oklahoma:   
        elif state == 'Oklahoma'.casefold():
            self.business_rule_pickle(classification_df, 'Oklahoma')


        #If the state is Colorado:  
        elif state == 'Colorado'.casefold():
            self.business_rule_pickle(classification_df, 'Colorado')

        #If enter other States:
        else:
            print('wrong state')
            break
    
    #if using grid search
    else:
        
        #If the state is Kansas:
        if state == 'Kansas'.casefold():
            #get the query result as dataframe and get the prediction result
            self.read_query_and_predict(classification_df, 'Kansas')
        
        #If the state is Minnesota:    
        elif state == 'Minnesota'.casefold():
            #get the query result as dataframe and get the prediction result
            self.read_query_and_predict(classification_df, 'Minnesota')

        else:
            print('wrong state')
            break 
    
    


# Functions

In [18]:
#define a function to get the subset of dataframe
def classification_subset_32(self, state):   #classification_subset_32
                                            #want the ab. version of state, not entire name
        
    '''
    This function subsets the dataframe to prepare classification.
    
    parameter:
    ---------
        state: Takes a string. The name of state to subset dataframe. 
        
    return:
    -------
        The subsetted dataframe.        
    '''

    #load the columns for subset from pickle:
    subset_columns = pickle.load(open('classification_columns.p', 'rb'))
    
    #subset the dataframe:
    classification_df = pd.DataFrame(self.data[subset_columns])
    
    classification_df = classification_df[classification_df['RTL_STATE_DSC'] == str(state).upper()]
    #state will pull from market_cd
    
    
    
    #create a LowPoint_Y column to store the result. 
    classification_df['LowPoint_Y'] = ''
    

In [ ]:
#define a function to run pickle for decision tree model. 
def decision_tree_pickle_32(self, classification_df, state):   #decision_tree_pickle_32
    
    '''
    This function takes the name of the state, and using the subsetted dataframe from function 'classification_subset' 
        to classify the stores using pickled decision tree model. 
        
    parameter:
    ----------
        classification_df: Takes a dataframe. The subsetted dataframe after excuting the function "classification_subset"
        state: Takes a string. The name of the State
        
    return:
    -------
        The dataframe with prediction result.        
    '''
    #call the binarize function to transform the columns into binary
    result_df = self.binarize([c for c in classification_df.columns if c not in ['RTL_STORE_CD',"LowPoint_Y"]])
    #check here.
    
    #load the model from pickle for that state
    filename = str(state).title()+'_decision_tree_model.p'
    decision_tree_model = pickle.load(open(filename, 'rb'))

    #prepare independent variables:
    X_labels = [c for c in result_df.columns if c not in [classification_df.columns]]
    X = result_df.loc[:,X_labels]

    #dependent variable:
    result_df['LowPoint_Y'] = decision_tree_model.predict(X) == 'LowPoint'
    

In [ ]:
#define a function to run pickle for business rule. 
def business_rule_pickle(self, classification_df, state):
    '''
    This function takes the name of the state, and using the subsetted dataframe from function 'classification_subset' 
        to classify the stores using pickled business rule.
    
    parameter:
    ----------
        state: Takes a string. The name of the State
        classification_df: Takes a dataframe. The subsetted dataframe after excuting the function "classification_subset"
        
    return:
    -------
        The dataframe with prediction result.        
    '''    

    #load the business rule from pickle for that State
    filename = str(state).title()+'_business_rule.p'
    rule_func = pickle.load(open('filename.p', 'rb'))

    #make the prediction
    rule_func(classification_df)

In [80]:
#define a function to get the sql query.
#Kansas: 162
#Utah:239
#Minnesota:177
#Corolado:125
#Oklahoma:201


def read_query_and_predict(self, classification_df, state):
    
    '''
    This function queries from Oracle SQL for specific state and store type and save the data in a dataframe. Then, use decision
        tree classifier to train the labeled data, and make prediction on whole dataset.
    
    parameter:
    ---------
        classification_df: Takes a dataframe. The subsetted dataframe after excuting the function "classification_subset"
        state: Takes a string. The state name
    
    return:
    -------
        a dataframe with prediction result. 
    '''
    
    
    #Access to Oracle DB
    host = 'tncluster6.cbi.net'
    port = 1521
    s_name = 'EDW1TS_EX.cbi.net'
    dsn_tns = cx_Oracle.makedsn(host, port, service_name = s_name)
    db_ts = cx_Oracle.connect('USERNAME', 'PASSWORD', dsn_tns)
    cursor = db_ts.cursor()
    
    if str(state) == 'Kansas'.casefold():
        state_code = str(162)
    elif str(state) == 'Utah'.casefold():
        state_code = str(239)
    elif str(state) == 'Minnesota'.casefold():
        state_code = str(177)
    elif str(state) == 'Corolado'.casefold():
        state_code = str(125)
    elif str(state) == 'Oklahoma'.casefold():
        state_code = str(201)
    
    query = '''select /*+ parallel(6) */ a15.STORE_CD  RTL_STORE_CD, 'NonLowPoint' as "BEERTYPE"
                from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
                join EDW.BI_CRN_D_ITEM_VW a12
                   on  (a11.ITEM_ID = a12.ITEM_ID)
                 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a13
                   on  (a11.DATE_ID = a13.REL_DATE_ID)
                 join EDW.BI_CRN_D_DISTRIBUTOR_VW a14
                   on  (a11.DIST_ID = a14.DIST_ID)
                 join EDW.BI_CRN_D_RETAILER_VW a15
                   on  (a11.RETAILER_ID = a15.RETAILER_ID)
                where (a13.REL_TIME_CD in ('L3_TY')
                 and a14.STATE_PROVINCE_COUNTRY_CD in (''' + "'" + state_code + "')" + ''' and a12.MASTER_SKU_CD not in ('80013450', '80013452', '80013455', '80013979', '80013456', '80013475', '80061325', '80013477', '80013982', '80013980', '80059842', '80058839', '80014020', '80014022', '80014023', '80014024', '80014025', '80013992', '80013991', '80013464', '80013466', '80013469', '80013471', '80013472', '80013473', '80013478', '80031998', '80013485', '80056926', '80058838', '80019746', '80031994', '80015966', '80014017', '80012704', '80014015', '80014009', '80014011', '80024732', '80013994', '80056172', '80014010'))
                group by a15.STORE_CD
                UNION 
                select /*+ parallel(6) */ a15.STORE_CD  RTL_STORE_CD,
                 'LowPoint' as "BEERTYPE"
                from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
                 join EDW.BI_CRN_D_ITEM_VW a12
                   on  (a11.ITEM_ID = a12.ITEM_ID)
                 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a13
                   on  (a11.DATE_ID = a13.REL_DATE_ID)
                 join EDW.BI_CRN_D_DISTRIBUTOR_VW a14
                   on  (a11.DIST_ID = a14.DIST_ID)
                 join EDW.BI_CRN_D_RETAILER_VW a15
                   on  (a11.RETAILER_ID = a15.RETAILER_ID)
                where (a13.REL_TIME_CD in ('L3_TY')
                 and a14.STATE_PROVINCE_COUNTRY_CD in (''' +"'" + state_code + "')" + ''' and a12.MASTER_SKU_CD not in ('80029020', '80060325', '80032234', '80032233', '80023100', '80031072', '80013435', '80013437', '80013439', '80057078', '80014016', '80013442', '80026873', '80056799', '80013444', '80014014', '80013447', '80013460', '80013457', '80013458', '80059543', '80013461', '80059542', '80059571', '80059572', '80059573', '80013982', '80013980', '80056922', '80014002', '80013968', '80029050', '80013515', '80013516', '80013517', '80013518', '80018933', '80056908', '80013520', '80014006', '80013993', '80013522', '80014026', '80013984', '80013978', '80013970', '80014012', '80014060', '80013986', '80013977', '80060330', '80060331', '80060328', '999', '80014008', '80013995', '80013972', '80056887', '80014001', '80031989', '80013985', '80013971', '80013983', '80014007', '80029021', '80015214', '80015215', '80014005', '80014004', '80013976', '80013990', '80013998', '80013989', '80013975', '80013997', '80027630', '80027631', '80027632', '80062390', '80062550', '80013996', '80014013', '80013988', '80013987', '80013974', '80013973', '80013999', '80014000', 'UNK', '80013981'))
                group by a15.STORE_CD '''
    
    try:
        cursor.execute(query)
        names = [ x[0] for x in cursor.description]
        rows = cursor.fetchall()
        StoreType = pd.DataFrame( rows, columns=names)

    finally:
        if cursor is not None:
            cursor.close()
            
    #If the above code runs sucessfully, it will return a dataframe which columns: store number and store type(LowPoint or NonLowPoint)
    
    
      
    #left join the store type data with subsetted dataframe
    result_df = pd.merge(classification_df, StoreType, on="RTL_STORE_CD", how="left")  
    
    #call the binarize function to transform the columns into binary
    result_df = self.binarize([c for c in classification_df.columns if c not in ['RTL_STORE_CD',"LowPoint_Y"]])
    
    #subset the training dataset
    train_df = result_df.loc[result_df['BEERTYPE'] != '']  
    
    #fit the model using traning data
    #independent variables:
    X_labels = [c for c in train_df.columns if c not in [classification_df.columns]]
    X_train = train_df.loc[:,X_labels]
    #dependent variable:
    Y_train = train_df.loc['BEERTYPE'] 
    
    #train the model, and find the best parameter
    parameters = {'max_depth':range(1,21), 'min_samples_leaf':range(3,21,3), 'min_samples_split':range(3,21,3), 'random_state': [0]}
    clf = tree.DecisionTreeClassifier()
    clf = GridSearchCV(clf, parameters, n_jobs = -2)
    clf.fit(X_train, Y_train)
    accuracy = clf.best_score_ 
    best_params = clf.best_params_
    best_depth = best_params['max_depth']
    best_leaf = best_params['min_samples_leaf']
    best_split = best_params['min_samples_split']

    #using the best parameter to train and fit the model
    clf = tree.DecisionTreeClassifier(max_depth=best_depth, min_samples_leaf= best_leaf, min_samples_split = best_split, random_state=0)
    clf = clf.fit(X_train,Y_train)

    #Prepare for prediction dataset
    X_pred = result_df.loc[:,X_labels]

    #make prediction for whole dataset.
    result_df['LowPoint_Y'] = clf.predict(X_pred) == 'LowPoint_Y'        
        
    #delete the column 'BEERTYPE'
    del result_df['BEERTYPE']    
    

#define a function to run pickle for business rule. 
def decision_tree_grid():
    
    '''
    This function uses the subsetted dataframe from 'classification_subset' function to classify the stores using grid search.
    
    parameter:
    ----------
        None
        
    return:
    -------
        The dataframe with prediction result.        
    '''      
    #merge the store type with original dataframe, and subset the training dataset.
    result_df = pd.merge(classification_df, StoreType, on="RTL_STORE_CD", how="left")  
    
    #call the binarize function to transform the columns into binary
    result_df = self.binarize([c for c in classification_df.columns if c not in ['RTL_STORE_CD',"LowPoint_Y"]])
    
    #subset the training dataset
    train_df = result_df.loc[result_df['BEERTYPE'] != '']  
    
    #fit the model using traning data
    #independent variables:
    X_labels = [c for c in train_df.columns if c not in [classification_df.columns]]
    X_train = train_df.loc[:,X_labels]
    Y_train = train_df.loc['BEERTYPE'] 
    
    #train the model, and find the best parameter
    parameters = {'max_depth':range(1,21), 'min_samples_leaf':range(3,21,3), 'min_samples_split':range(3,21,3), 'random_state': [0]}
    clf = tree.DecisionTreeClassifier()
    clf = GridSearchCV(clf, parameters, n_jobs = -2)
    clf.fit(X_train, Y_train)
    accuracy = clf.best_score_ 
    best_params = clf.best_params_
    best_depth = best_params['max_depth']
    best_leaf = best_params['min_samples_leaf']
    best_split = best_params['min_samples_split']

    #using the best parameter to train the model
    clf = tree.DecisionTreeClassifier(max_depth=best_depth, min_samples_leaf= best_leaf, min_samples_split = best_split, random_state=0)
    clf = clf.fit(X_train,Y_train)

    #Prepare for prediction dataset
    X_pred = result_df.loc[:,X_labels]

    #make prediction for whole dataset.
    result_df['LowPoint_Y'] = clf.predict(X_pred) == 'LowPoint_Y'        
        
    #delete the column 'BEERTYPE'
    del result_df['BEERTYPE']    
    

In [ ]:
#automate the process for classification 
def classify_32(self, state, isPickle):
    
    '''
    This function takes in dataframe, and classify the store type to either LowPoint store or NonLowPoint store.
        LowPoint: stores that can only sell 3.2% beer.
        NonLowPoint: stores that can sell any beer.
    
    Parameter:
    ---------
        state: takes a string. The name of state.
        isPickle: takes a boolean. Indicates classification method. 'True' for pickle, 'False' for grid search
        
    Return:
    -------
        the dataframe with prediction result under 'LowPoint_Y' column.
        
    '''
    #classification_subset 
    classification_df = self.classification_subset(self.data, state)
    
    #if using Pickle
    if isPickle:
        
        #Decision tree model for Kansas and Minnesota
        #If the state is Kansas:
        if state == 'Kansas'.casefold():
            self.decision_tree_pickle(classification_df, 'Kansas')

        #If the state is Minnesota:   
        elif state == 'Minnesota'.casefold():
            self.decision_tree_pickle(classification_df, 'Minnesota')
      
    
        #Business rule for Utah, Oklahoma, and Colorado
        #If the state is Utah:
        elif state == 'Utah'.casefold():
            self.business_rule_pickle(classification_df, 'Utah')
            
        #If the state is Oklahoma:   
        elif state == 'Oklahoma'.casefold():
            self.business_rule_pickle(classification_df, 'Oklahoma')


        #If the state is Colorado:  
        elif state == 'Colorado'.casefold():
            self.business_rule_pickle(classification_df, 'Colorado')

        #If enter other States:
        else:
            print('wrong state')
            break
    
    #if using grid search
    else:
        
        #If the state is Kansas:
        if state == 'Kansas'.casefold():
            #get the query result as dataframe and get the prediction result
            self.read_query_and_predict(classification_df, 'Kansas')
        
        #If the state is Minnesota:    
        elif state == 'Minnesota'.casefold():
            #get the query result as dataframe and get the prediction result
            self.read_query_and_predict(classification_df, 'Minnesota')

        else:
            print('wrong state')
            break 
    

# SQL

In [49]:
!pip install cx_Oracle

    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\qguo\AppData\Local\Temp\pip-build-s_8c603c\cx-Oracle\setup.py", line 174, in <module>
        raise DistutilsSetupError("cannot locate an Oracle software " \
    distutils.errors.DistutilsSetupError: cannot locate an Oracle software installation
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\qguo\AppData\Local\Temp\pip-build-s_8c603c\cx-Oracle\


In [ ]:
import pyodbc

# Specifying the ODBC driver, server name, database, etc. directly
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=testdb;UID=me;PWD=pass')

# Using a DSN, but providing a password as well
cnxn = pyodbc.connect('DSN=test;PWD=password')

# Create a cursor from the connection
cursor = cnxn.cursor()

In [48]:
import cx_Oracle
con = cx_Oracle.connect('amartinezcotto/Kohlia#743@tncluster6.cbi.net/EDW1TS_EX.cbi.net')
cur = con.cursor()
cur.execute('''select /*+ parallel(6) */ a15.STORE_CD  RTL_STORE_CD, 'True' as "NonLowPoint_Y"
from EDW.BI_CRN_F_RETAIL_DEPL_MVW_VW a11
 join EDW.BI_CRN_D_ITEM_VW a12
 on (a11.ITEM_ID = a12.ITEM_ID)
 join EDW.BI_CRN_D_REL_TIME_DETAIL_VW a13
 on (a11.DATE_ID = a13.REL_DATE_ID)
 join EDW.BI_CRN_D_DISTRIBUTOR_VW a14
 on (a11.DIST_ID = a14.DIST_ID)
 join EDW.BI_CRN_D_RETAILER_VW a15
 on (a11.RETAILER_ID = a15.RETAILER_ID)
where (a13.REL_TIME_CD in ('L3_TY')
 and a14.STATE_PROVINCE_COUNTRY_CD in ('162')
 and a12.MASTER_SKU_CD not in ('80013450', '80013452', '80013455', '80013979', '80013456', '80013475', '80061325', '80013477', '80013982', '80013980', '80059842', '80058839', '80014020', '80014022', '80014023', '80014024', '80014025', '80013992', '80013991', '80013464', '80013466', '80013469', '80013471', '80013472', '80013473', '80013478', '80031998', '80013485', '80056926', '80058838', '80019746', '80031994', '80015966', '80014017', '80012704', '80014015', '80014009', '80014011', '80024732', '80013994', '80056172', '80014010'))
group by a15.STORE_CD''')
               
row = cur.fetchone()
       
cur.close()
con.close()

ModuleNotFoundError: No module named 'cx_Oracle'

In [46]:
#sql notes
#Kansas: 162
#Utah:239
#Minnesota:177
#Corolado:125
#Oklahoma:201


# Kansas decision tree model

In [ ]:
parameters = {'max_depth':range(1,21), 'min_samples_leaf':range(3,21,3), 'min_samples_split':range(3,21,3), 'random_state': [0]}
clf = tree.DecisionTreeClassifier()
clf = GridSearchCV(clf, parameters, n_jobs = -2)
clf.fit(X, Y)
accuracy = clf.best_score_ 
best_params = clf.best_params_
best_depth = best_params['max_depth']
best_leaf = best_params['min_samples_leaf']
best_split = best_params['min_samples_split']

clf = tree.DecisionTreeClassifier(max_depth=best_depth, min_samples_leaf= best_leaf, min_samples_split = best_split, random_state=0)
clf = clf.fit(X,Y)
# save the model to disk
filename = 'Kansas_decision_tree_model.p'
pickle.dump(clf, open(filename, 'wb'))

# Minnesota decision tree model

In [ ]:
parameters = {'max_depth':range(1,21), 'min_samples_leaf':range(3,21,3), 'min_samples_split':range(3,21,3), 'random_state': [0]}
clf = tree.DecisionTreeClassifier()
clf = GridSearchCV(clf, parameters, n_jobs = -2)
clf.fit(X, Y)
accuracy = clf.best_score_ 
best_params = clf.best_params_
best_params = clf.best_params_
best_depth = best_params['max_depth']
best_leaf = best_params['min_samples_leaf']
best_split = best_params['min_samples_split']

clf = tree.DecisionTreeClassifier(max_depth=best_depth, min_samples_leaf= best_leaf, min_samples_split = best_split, random_state=0)
clf = clf.fit(X,Y)
# save the model to Pickle
pickle.dump(clf, open('Minnesota_decision_tree_model.p', 'wb'))

# Utah business rule

In [38]:
def Rules_Utah(dataframe):

    for row in range(len(dataframe)):
        if dataframe.loc[row,'RTL_CHANNEL_DSC'] == 'LIQUOR' or 'MILITARY' in dataframe.loc[row,'RTL_CHANNEL_DSC']:
            
            dataframe.loc[row,'LowPoint_Y'] = False
        else:
            
            dataframe.loc[row,'LowPoint_Y'] = True
    
    return dataframe

#save the Utah business rule into pickle
pickle.dump(Rules_Utah, open('Utah_business_rule.p', 'wb'))

# Oklahoma business rule

In [ ]:
def Rules_Oklahoma(dataframe):
    result_list = []
    for row in range(len(dataframe)):
        if dataframe.loc[row,'RTL_CHANNEL_DSC'] == 'DISTRIBUTOR/SUB-DISTRIBUTOR' or dataframe.loc[row,'RTL_CHANNEL_DSC'] == 'EXTENDED MASTER OFF-PREMISE':
                result_list.append(False)
        else:
                result_list.append(True)
    return result_list

#save the Oklahoma business rule into pickle
pickle.dump(Rules_Oklahoma, open('Oklahoma_business_rule.p', 'wb'))

# Corolado business rule

In [41]:
def Rules_Corolado(dataframe):
    for row in range(len(dataframe)):
        if dataframe.loc[row,'RTL_PREMISE_TYPE_CD'] == 'OFF':
            if dataframe.loc[row,'RTL_CHANNEL_DSC'] == 'LIQUOR' or dataframe.loc[row,'RTL_CHANNEL_DSC'] == 'DRUG' or dataframe.loc[row,'RTL_CHANNEL_DSC'] == 'MILITARY OFF-PREMISE':
                dataframe.loc[row,'LowPoint_Y'] = False
            elif dataframe.loc[row,'RTL_CHANNEL_DSC'] == 'GROCERY':
                if dataframe.loc[row,'RTL_LIQUOR_FLG'] == 'Y':
                    dataframe.loc[row,'LowPoint_Y'] = False
                else:
                    dataframe.loc[row,'LowPoint_Y'] = True
            else:
                dataframe.loc[row,'LowPoint_Y'] = True
        else:
             dataframe.loc[row,'LowPoint_Y'] = False
    return dataframe

#save the Corolado business rule into pickle
pickle.dump(Rules_Corolado, open('Corolado_business_rule.p', 'wb'))

# The End for the function

In [ ]:
#function for production for grid search
def decision_tree_prediction_grid(self, storeType, binary_column, delete_column):

    '''
    Use decision tree classification model with grid search to train and predict for input dataframe.
    06/23/2017
    
    parameter:
    ----------
        
        storeType: takes a dataframe. Data with stores and their types. (LowPoint and NonLowPoint)
                   LowPoint: stores that can only sell 3.2% beer.
                   NonLowPoint: stores that can sell any beer.
        
        binary_column: takes a string or a list of string as input. The input must be a column name present in the data. 
                       The names of columns need to be binarized.
    
        delete_Column: takes a string or a list of string as input. The input must be a column name present in the data. 
                       The names of the columns that don't want to be included when trainning. 
    
    return:
    -------
    
        The original dataframe with the prediction results under colomn 'PREDICTION'
        'PREDICTION' has two values: 
                                    LowPoint: stores that can only sell 3.2% beer.
                                    NonLowPoint: stores that can sell any beer.
    
    '''    
    
    #merge the data with their label of store type. 
    FullData = pd.merge(self.data, StoreType, on="RTL_STORE_CD", how="left")
    FullData.loc[FullData['BEERTYPE'].isnull(),"BEERTYPE"] = "LowPoint"
    
    #call binarize function to convert columns into binary. 
    binarized_df = self.binarize(binary_column)
    
    #train full data with cross validation
    #independent variables:
    X_labels = [c for c in binarized_df.columns if c not in delete_column]
    X = binarized_df.loc[:,X_labels]
    Y = binarized_df['BEERTYPE']
    
    #train and fit the model
    parameters = {'max_depth':range(1,21), 'min_samples_leaf':range(3,21,3), 'min_samples_split':range(3,21,3), 'random_state': [0]}
    clf = tree.DecisionTreeClassifier()
    clf = GridSearchCV(clf, parameters, n_jobs = -2)
    clf.fit(X, Y)
    accuracy = clf.best_score_ 
    best_params = clf.best_params_
    best_depth = best_params['max_depth']
    best_leaf = best_params['min_samples_leaf']
    best_split = best_params['min_samples_split']
    
    #use the trained model to make prediction
    clf = tree.DecisionTreeClassifier(max_depth=best_depth, min_samples_leaf= best_leaf, min_samples_split = best_split, random_state=0)
    clf = clf.fit(X,Y)
    #X in fit() is wrong, need the whole data
    self.data['LowPoint_Y'] = clf.predict(X) ==
    
  

# Pickle

In [ ]:
#function for production pickle
def decision_tree_prediction(self, delete_column):
    '''
    Use decision tree classification model that was saved in pickle to predict for input dataframe. This function is used after
    function 'binarize' is processed. 
    06/23/2017
    
    parameter:
    ----------
    
        delete_Column: takes a string or a list of string as input. The input must be a column name present in the data. 
                       The names of the columns that don't want to be included in trainning. 
    
    return:
    -------
    
        The original dataframe with the prediction results under colomn 'PREDICTION'
        'PREDICTION' has two values: 
                                    LowPoint: stores that can only sell 3.2% beer.
                                    NonLowPoint: stores that can sell any beer.
    
    '''
    #create a result column first
    
    #load the model saved in pickle
    decision_tree_model = pickle.load(open('decision_tree_model.sav', 'rb'))
    
    #independent variables:
    X_labels = [c for c in self.data.columns if c not in delete_column]
    X = self.data.loc[:,X_labels]
    
    #dependent variable:
    self.data['PREDICTION'] = decision_tree_model.predict(X)
    
    #decision_tree_model.predict(X) == 'LowPoint'